In [1]:
from bs4 import BeautifulSoup
import pandas as pd 
import requests
import urllib.request


In [2]:
base_url = 'https://www.hanbit.co.kr'
sub_url = '/academy/books/new_book_list.html'
url = base_url + sub_url
with urllib.request.urlopen(url) as response:
    html = response.read()
    soup = BeautifulSoup(html, 'html.parser')

In [3]:
# 덩어리로 한페이지에 몇개있는지 
lis = soup.select('.sub_book_list')
len(lis)

20

In [5]:
# 세부 들어갈 url주소찾기
book_url = lis[0].find('a').attrs['href']
book_url

'/academy/books/book_view.html?p_code=B4797877052'

In [6]:
with urllib.request.urlopen(base_url+book_url) as response:
    html = response.read()
    book_soup = BeautifulSoup(html, 'html.parser')

In [7]:
info = book_soup.select_one('.store_product_info_box')
title = info.find('h3').string
title

'포르잔 C++ 에센셜'

In [13]:
#꼭 딕셔너리로 안보고,바로 리스트의 정보값만 추출해도된다.
# 그러나 키값이 없는경우를 대비하여 딕셔너리 형태로.
lis = info.find_all('li')

book_dict = {}
for li in lis:
    idx = li.find('strong').get_text().find(':')
    key = li.find('strong').get_text()[:idx-1]
    value = li.find('span').get_text()
    book_dict.update([[key,value]])
book_dict

{'저자': 'Behrouz A. Forouzan , Richard F. Gilberg ',
 '번역': '윤인성 ',
 '출간': '2020-11-10',
 '페이지': '756 쪽',
 'ISBN': '9791156645061',
 '물류코드': '4506'}

In [16]:
#저자 처리
al = 'Behrouz A. Forouzan , Richard F. Gilberg'.split(',')
al =list(map(lambda s: s.rstrip(), al))
','.join(al)

'Behrouz A. Forouzan, Richard F. Gilberg'

In [17]:
#포문식 정리1
#저자
info = book_soup.select_one('.store_product_info_box')
book_lis = info.find_all('li')
al = book_lis[0].find('span').get_text().split(',')
authors = ','.join(list(map(lambda s: s.rstrip(), al)))
#다른정보는 없을수있으므로 조건식으로 
for bli in book_lis[1:]:
     key = bli.find('strong').get_text()
     if key.find('번역') >= 0:
         tl = bli.find('span').get_text().split(',')
         translators = ','.join(list(map(lambda s: s.rstrip(), tl)))
     elif key.find('출간') >= 0:
         date = bli.find('span').get_text()
     elif key.find('페이지') >= 0:
         page = int(bli.find('span').get_text()[:-2])
     elif key.find('ISBN') >= 0:
         isbn = bli.find('span').get_text()
         
print(authors, translators, date, page, isbn)

Behrouz A. Forouzan, Richard F. Gilberg 윤인성 2020-11-10 756 9791156645061


In [18]:
pb_ps = book_soup.select_one('.payment_box').find_all('p')
list_prices = []; sales_prices = []; mileages = []

idx = pb_ps[0].select_one('.pbr').get_text().find('원')
lp = pb_ps[0].select_one('.pbr').get_text()[:idx]
lp = int(lp.replace(',',''))

idx = pb_ps[1].select_one('.pbr').get_text().find('원')
sp = pb_ps[1].select_one('.pbr').get_text()[:idx]
sp = int(sp.replace(',',''))

idx = pb_ps[2].select_one('.pbr').get_text().find('점')
mile = pb_ps[2].select_one('.pbr').get_text()[:idx]
mile = int(mile.replace(',',''))

print(lp, sp, mile)

32000 32000 960
